In [1]:
# python manage.py shell_plus --notebook
import os
import django
from django.core.files.uploadedfile import UploadedFile
from wfrp.models import Campaign, Item, Talent, Spell, PlayableCharacter
from users.models import User

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

### Database initialization

In [2]:
# Defining party users
users = [{'name': 'ricardo'},
         {'name': 'danna'},
         {'name': 'omar'},
         {'name': 'caceres'},
         {'name': 'aleja'},
         {'name': 'elliot'},
         {'name': 'felipe'},
         {'name': 'velez'}]

for user in users:
  user['password'] = f'{user["name"]}_changeme'

current_users = [user.username for user in User.objects.all()]

for user in users:
  if user['name'] not in current_users:
    user_instance = User.objects.create_user(username=user['name'], 
                                             password=user['password'])
    user_instance.save()
    print(user['name'], 'added successfully!')

for user in users:
  user['instance'] = User.objects.get(username=user['name'])

GUNNAR = {
  'user': User.objects.get(username='ricardo'),
  'name': 'Gunnar Hrolfsson',
  'species': 'Dwarf',
  'character_class': 'Warrior',
  'career': 'Slayer',
  'career_level': 'Giant Slayer',
  'career_path': 'Apprentice Artisan, Troll Slayer, Giant Slayer',
}

MOLRELLA = {
  'user': User.objects.get(username='felipe'),
  'name': 'Molrella Brandysnap',
  'species': 'Halfling',
  'character_class': 'Rogue',
  'career': 'Slayer',
  'career_level': 'Giant Slayer',
  'career_path': 'Pauper, Prowler, Thief',
}

FERDINAND = {
  'user': User.objects.get(username='caceres'),
  'name': 'Ferdinand Gruber',
  'species': 'Human',
  'character_class': 'Academic',
  'career': 'Wizard',
  'career_level': "Wizard's Apprentice",
  'career_path': "Scion, Wizard's Apprentice",
}

AMRIS = {
  'user': User.objects.get(username='omar'),
  'name': 'Amris Emberfell',
  'species': 'High Elf',
  'character_class': 'Burgher',
  'career': 'Wizard',
  'career_level': "Merchant",
  'career_path': "Scion, Wizard's Apprentice",
}

ELSE = {
  'user': User.objects.get(username='danna'),
  'name': 'Else Sigloben',
  'species': 'Human',
  'character_class': 'Ranger',
  'career': 'Witch Hunter',
  'career_level': "Witch Hunter",
  'career_path': "Interrogator, Witch Hunter",
}

YAPER = {
  'user': User.objects.get(username='elliot'),
  'name': 'Yaper Dio',
  'species': 'Human',
  'character_class': 'Rogue',
  'career': 'Child',
  'career_level': "Child Hero Apprentice",
  'career_path': "Poor Child, Child Hero Apprentice", 
}

PARTY = [GUNNAR, MOLRELLA, FERDINAND, AMRIS, ELSE, YAPER]


ricardo added successfully!
danna added successfully!
omar added successfully!
caceres added successfully!
aleja added successfully!
elliot added successfully!
felipe added successfully!
velez added successfully!


In [3]:
campaign_master = User.objects.get(username='velez')
campaign_name = 'Haciendo la ronda'
campaign_description = '''
Los Personajes, que paseaban por el mercado de Ubersreik, se ven metidos en un tumulto y, más tarde, se les acusa de instigarlo. Misteriosamente, una abogada local interviene y convence al juez responsable del caso de que permita a los Personajes que paguen su ‘deuda’ trabajando como miembros de la Guardia.

Pronto, el grupo se ve patrullando las calles de Ubersreik con Rudi Klumpenklug, un sargento de la guardia absolutamente corrupto. Los guardias temporales se ven expuestos a diversos delitos, ninguno de los cuales Klumpenklug muestra interés alguno por perseguir, dejando que los Personajes resuelvan, ignoren o incluso exploten cada situación como prefieran.

Por último, Ilse Fassenwütend se pone en contacto con el grupo. Fassenwütend es una guarda de caminos, que afirma ser capaz de hacer que la sentencia de los Personajes sea conmutada si le ayudan a escoltar a un criminal al lugar donde debe ser ejecutado.

Sin embargo, no será tarea fácil. El grupo tendrá ante sí una noche horrorosa, llevando a un hombre aterrorizado a través de unas calles extrañamente alteradas mientras Morrslieb, la luna del Caos, está en cuarto creciente (y por lo tanto brilla en el cielo) y sectarios mutantes los atacan por todos lados ¿Conseguirán los Personajes proteger al hombre, que afirma ser inocente, para luego llevarlo hasta el cadalso? ¿O elegirán un camino distinto?
'''

if len(Campaign.objects.filter(name=campaign_name)) == 0:
  campaign = Campaign(master=campaign_master, 
                      name=campaign_name,
                      description=campaign_description.strip())
  campaign.save()
  campaign.cover_image.save(f'ubersreik.jpg', UploadedFile(file=open(f'../example_media/ubersreik.jpg', 'rb'), content_type='image/png'))
else:
  campaign = Campaign.objects.get(name=campaign_name)

In [4]:
for character in PARTY:
  similar_characters = PlayableCharacter.objects.filter(name=character['name']) 
  character_instance = similar_characters[0] if len(similar_characters) > 0 else None
  if character_instance not in PlayableCharacter.objects.all():
    character_instance = PlayableCharacter(**character, campaign = campaign)
    character_instance.save()
    character_name = character_instance.name.split(" ")[0].lower()
    character_instance.character_avatar.save(f'{character_name}.png', UploadedFile(file=open(f'../example_media/{character_name}.png', 'rb'), content_type='image/png'))

In [6]:
import pandas as pd

char_data = pd.read_excel('../example_media/initial_characters.xlsx')
char_data = char_data.to_dict(orient='list')
char_data = {key.strip():value[0] for key,value in char_data.items()}

gunnar = PlayableCharacter.objects.filter(name='Gunnar Hrolfsson')
gunnar.update(**char_data)

1